# Import Libs

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, balanced_accuracy_score, precision_recall_fscore_support, recall_score, precision_score, f1_score
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Preprocessing

Define mark which will  set min mark limit

In [ ]:
data = pd.read_excel('dataset.xlsx')
data.dropna

min_mark = 4

In [ ]:
import matplotlib.pyplot as plt

x = range(len(data['Средний балл']))
y = data['Средний балл']

plt.scatter(x, y)
plt.grid()
plt.show()

In [ ]:
# Replase mark of student by belonging to the class of successfully passed
data['Сдал'] = pd.cut(x=data['Средний балл'], bins=[0, min_mark, 10], labels=[0, 1])
data.drop(['Средний балл'], axis = 1, inplace= True)

# Drop indicators which are not connected with extracurricular activities
data.drop([ 'Учащийся',
            'Дата прохождения теста',
            '15.Образование Ваших родителей?', 
            '3.С какими оценками Вы закончили школу?',
            '4.Ходили ли Вы на подготовительные курсы перед поступлением в вуз?',
            '7.Какая у Вас семья?',
            '10.Получали ли Вы стипендию? (в течение последнего года)',
            '11.Оцените, как Вам нравится учиться?'],
            axis = 1, inplace = True)
            
data.head()

Lets find out how many students did not pass the exams

In [ ]:
passed = 0
not_passed = 0
res = 0
for i in data['Сдал']:
    if i == 1:
        passed += 1
    elif i == 0:
        not_passed += 1
    res += 1
print(f"TOTAL: {res}\nPassed: {passed}\nNot pas: {not_passed}")

Split our data in 3 parts for train, test and validation

In [ ]:
def train_test_val_split(train_ratio=0.75, test_ratio=0.10, validation_ratio=0.15):
    label_encoder = LabelEncoder()

    s = (data.dtypes == 'object')
    object_cols = list(s[s].index)
    label_data = data.copy()
    for col in object_cols:
        label_encoder.fit(data[col])
        label_data[col] = label_encoder.transform(data[col])
    X_train, X_test, y_train, y_test = train_test_split(label_data.drop('Сдал', axis = 1), label_data['Сдал'], test_size=1 - train_ratio, random_state=1)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=test_ratio/(test_ratio + validation_ratio), random_state=1)
    return X_train, X_test, X_val, y_train, y_test, y_val

In [ ]:
#will be replaced by train_test_val_split()
def evel_split():
    train = data.loc[:346, :]
    val = data.loc[347:, :].drop('Сдал', axis = 1)

    s = (train.dtypes == 'object')
    object_cols = list(s[s].index)


    label_train = train.copy()
    label_val = val.copy()

    label_encoder = LabelEncoder()

    for col in object_cols:
        label_encoder.fit(data[col])
        label_train[col] = label_encoder.transform(train[col])
        label_val[col] = label_encoder.transform(val[col])

    from sklearn.model_selection import train_test_split
    X_train, X_test, y_train, y_test = train_test_split(label_train.drop('Сдал', axis = 1), 
                                                        label_train['Сдал'], test_size=0.3, random_state=42)
    return X_train, X_test, y_train, y_test

In [ ]:
X_train, X_test, y_train, y_test = evel_split()
#X_train, X_test, X_val, y_train, y_test, y_val = train_test_val_split(train_ratio=0.75, test_ratio=0.10, validation_ratio=0.15)

# Model variants
In that part I will choose the classification model which will predict if student pass exams successfully or not

## Metrics Functions

In [ ]:
def classification_quality(y_test, y_pred):
    print("Accuracy:", accuracy_score(y_test, y_pred))
    print("Recall:", recall_score(y_test, y_pred, average='macro'))
    print("Precision:", precision_score(y_test, y_pred, average='macro'))
    print("F1:", f1_score(y_test, y_pred, average='macro', labels=np.unique(y_pred)))
    print("Weighted Recall:", (precision_recall_fscore_support(y_test, y_pred, average='macro')))

In [ ]:
def regression_quality(y_test, y_pred):
    print("MSE:", mean_squared_error(y_test, y_pred))
    print("RMSE:", mean_squared_error(y_test, y_pred)**(1/2))
    print("MAE:", mean_absolute_error(y_test, y_pred))
    print("R2:", r2_score(y_test, y_pred))

In [ ]:
metrics_list = ['Accuracy', 'Balanced Accuracy', 'Recall', 'Precision', 'F1', 'MSE', 'RMSE', 'MAE', 'R2']
model_array = []
output_array = []

def compilance_print(model, y_test, y_pred, model_flag):
    temp_array = []
    if model_flag == 'cls': # classifier
        temp_array.append(accuracy_score(y_test, y_pred))
        temp_array.append(balanced_accuracy_score(y_test, y_pred))
        temp_array.append(recall_score(y_test, y_pred, average='macro'))
        temp_array.append(precision_score(y_test, y_pred, average='macro'))
        temp_array.append(f1_score(y_test, y_pred, average='macro', labels=np.unique(y_pred)))
        for i in range(4): temp_array.append(None)
    elif model_flag == 'reg': # regression
        for i in range(5): temp_array.append(None)
        temp_array.append(mean_squared_error(y_test, y_pred))
        temp_array.append(mean_squared_error(y_test, y_pred)**(1/2))
        temp_array.append(mean_absolute_error(y_test, y_pred))
        temp_array.append(r2_score(y_test, y_pred))
    else: 
        print('Error')
        for i in range(len(metrics_list)): temp_array.append(None)

    model_indx = -1
    flag = 1
    for indx, _model in enumerate(model_array):
        if _model == model:
            model_indx = indx
            flag = 0
    if flag:
        output_array.append([0]* len(metrics_list))
        model_indx = len(model_array)
        model_array.append(model)
    for indx, el in enumerate(temp_array):
        output_array[model_indx][indx] = el

In [ ]:
def graph_show(model, X_test, y_test):
    metrics.plot_confusion_matrix(model, X_test, y_test) 
    metrics.plot_roc_curve(model, X_test, y_test)
    plt.show()

## Classifier

### Perception Classifier Model

In [ ]:
from sklearn.linear_model import Perceptron
Perc = Perceptron()
Perc.fit(X_train, y_train)

y_pred = Perc.predict(X_test)

classification_quality(y_test, y_pred)
compilance_print('Perceptron', y_test, y_pred, 'cls')


### Random Forest Classifier Model

In [ ]:
from sklearn.ensemble import RandomForestClassifier
RanF = RandomForestClassifier()
RanF.fit(X_train, y_train)

y_pred = RanF.predict(X_test)

classification_quality(y_test, y_pred)
compilance_print('RandomForest', y_test, y_pred, 'cls')


### Gradient Boosting Classifier Model

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
GraB = GradientBoostingClassifier()
GraB.fit(X_train, y_train)

y_pred = GraB.predict(X_test)

classification_quality(y_test, y_pred)
compilance_print('GradientBoosting', y_test, y_pred, 'cls')

### MultinomialNB Classifier Model

In [ ]:
from sklearn.naive_bayes import MultinomialNB
MulNB = MultinomialNB()
MulNB.fit(X_train, y_train)

y_pred = MulNB.predict(X_test)

classification_quality(y_test, y_pred)
compilance_print('MultinomialNB', y_test, y_pred, 'cls')

### ComplementNB Classifier Model

In [ ]:
from sklearn.naive_bayes import ComplementNB
ComNB = ComplementNB()
ComNB.fit(X_train, y_train)

y_pred = ComNB.predict(X_test)

classification_quality(y_test, y_pred)
compilance_print('ComplementNB', y_test, y_pred, 'cls')

### GaussianNB Classifier Model

In [ ]:
from sklearn.naive_bayes import GaussianNB
GaNB = GaussianNB()
GaNB.fit(X_train, y_train)

y_pred = GaNB.predict(X_test)

classification_quality(y_test, y_pred)
compilance_print('GaussianNB', y_test, y_pred, 'cls')

### CategoricalNB Classifier Model

In [ ]:
from sklearn.naive_bayes import CategoricalNB
CatNB = CategoricalNB()
CatNB.fit(X_train, y_train)

y_pred = CatNB.predict(X_test)

classification_quality(y_test, y_pred)
compilance_print('CategoricalNB', y_test, y_pred, 'cls')

### BernoulliNB Classifier Model

In [ ]:
from sklearn.naive_bayes import BernoulliNB
BerNB = BernoulliNB()
BerNB.fit(X_train, y_train)

y_pred = BerNB.predict(X_test)

classification_quality(y_test, y_pred)
compilance_print('BernoulliNB', y_test, y_pred, 'cls')

### SVC Classifier Model

In [ ]:
from sklearn.svm import SVC
svc = SVC()
svc.fit(X_train, y_train)

y_pred = svc.predict(X_test)

classification_quality(y_test, y_pred)
compilance_print('SVC', y_test, y_pred, 'cls')

### LinearSVC Classifier Model

In [ ]:
from sklearn.svm import LinearSVC
lsvc = LinearSVC()
lsvc.fit(X_train, y_train)

y_pred = lsvc.predict(X_test)

classification_quality(y_test, y_pred)
compilance_print('LinearSVC', y_test, y_pred, 'cls')

### OneClassSVM Classifier Model

In [ ]:
from sklearn.svm import OneClassSVM
oclsvc = OneClassSVM()
oclsvc.fit(X_train, y_train)

y_pred = oclsvc.predict(X_test)

classification_quality(y_test, y_pred)
#compilance_print('OneClassSVM', y_test, y_pred, 'cls')

### DecisionTreeClassifier Model

In [ ]:
from sklearn.tree import DecisionTreeClassifier
dtr = DecisionTreeClassifier()
dtr.fit(X_train, y_train)

y_pred = dtr.predict(X_test)

classification_quality(y_test, y_pred)
compilance_print('DecisionTreeClassifier', y_test, y_pred, 'cls')

### ExtraTreeClassifier Model

In [ ]:
from sklearn.tree import plot_tree, ExtraTreeClassifier
extr = ExtraTreeClassifier()
extr.fit(X_train, y_train)

y_pred = extr.predict(X_test)

#plot_tree(extr)
classification_quality(y_test, y_pred)
compilance_print('ExtraTreeClassifier', y_test, y_pred, 'cls')

### MLPClassifier Model

In [ ]:
from sklearn.neural_network import MLPClassifier
MLPCl = MLPClassifier()
MLPCl.fit(X_train, y_train)

y_pred = MLPCl.predict(X_test)

classification_quality(y_test, y_pred)
compilance_print('MLPClassifier', y_test, y_pred, 'cls')

## Matrix
That table helps to understand which model is better for my work

In [ ]:
pd.DataFrame(index=metrics_list, columns=model_array, data=np.array(pd.DataFrame(output_array).T))

I choose model X due to its statistics

In [ ]:
my_model = MLPCl

# Validation

In [ ]:
data = pd.read_excel('dataset.xlsx')
students = data[347:].drop('Средний балл', axis = 1)

In [ ]:
students.loc[:, 'Сдал'] = my_model.predict(label_val)
students.index += 1 

In [ ]:
students[students['Сдал'] == 0]['Учащийся']